In [ ]:
# General imports
import os
import glob
import healpy
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.io.fits.header import Header
from astropy.table import Table, Column, vstack

# SHE_PPT imports
from SHE_PPT import products
from SHE_PPT import file_io
from SHE_PPT.table_formats.shear_estimates import tf as setf

# Make the plots interactive
%matplotlib notebook

In [ ]:
def select_true_universe_sources(catalog_file_names, ra_range, dec_range):
    """ Loads all the True Universe catalog files and selects only those
    sources that fall inside the specified (RA, Dec) region.

    """
    # Loop over the True Universe catalog files and select the relevant sources
    merged_catalog = None

    for file_name in catalog_file_names:
        # Load the catalog table
        catalog = Table.read(file_name, format="fits")

        # Get the (RA, Dec) columns
        ra = catalog["ra_mag"] if "ra_mag" in catalog.colnames else catalog["RA"]
        dec = catalog["dec_mag"] if "dec_mag" in catalog.colnames else catalog["DEC"]

        # Check which sources fall inside the given (RA, Dec) ranges
        cond_ra = np.logical_and(ra > ra_range[0], ra < ra_range[1])
        cond_dec = np.logical_and(dec > dec_range[0], dec < dec_range[1])
        cond = np.logical_and(cond_ra, cond_dec)

        if np.any(cond):
            # Add the selected sources to the merged catalog
            if merged_catalog is None:
                merged_catalog = catalog[cond]
            else:
                merged_catalog = vstack([merged_catalog, catalog[cond]])

    return merged_catalog

In [ ]:
# Get the True Universe star and galaxy catalog file names

star_catalog_product_filename = "/mnt/cephfs/share/SC456/SIM-VIS/vis_science_T11/StarCatT11.xml"
galaxy_catalog_product_filename = "/mnt/cephfs/share/SC456/SIM-VIS/vis_science_T11/GalCatT11.xml"

star_catalog_file_names = file_io.read_xml_product(star_catalog_product_filename).get_data_filenames()
galaxy_catalog_file_names = file_io.read_xml_product(galaxy_catalog_product_filename).get_data_filenames()

In [ ]:
# Set the metadata about this batch
field_number = 52929
batch_number = 0

In [ ]:
# Set the path to the working directory
workdir = "/mnt/cephfs/brg/sc4-workdir"

shear_estimates_product = file_io.read_xml_product(os.path.join(workdir,
                                                                "combined_shear_estimates.xml"))

In [ ]:
# Load the tables for each shear estimation method

methods = ("BFD","KSB","LensMC","MomentsML","REGAUSS")
shear_tables = {}

for method in methods:
    fn = shear_estimates_product.get_method_filename(method)
    if fn is None:
        shear_tables[method] = None
    else:
        shear_tables[method] = Table.read(os.path.join(workdir,fn))

In [ ]:
# Determine the ra/dec range covered by this file

ra_range = np.array((1e99,-1e99))
dec_range = np.array((1e99,-1e99))

for method in methods:
    shear_table = shear_tables[method]
    if shear_table is None:
        continue
    ra_col = shear_table[setf.x_world]
    dec_col = shear_table[setf.x_world]
    
    ra_range[0] = np.min((ra_range[0],np.min(ra_col.data)))
    ra_range[1] = np.max((ra_range[1],np.max(ra_col.data)))
    
    dec_range[0] = np.min((dec_range[0],np.min(dec_col.data)))
    dec_range[1] = np.max((dec_range[1],np.max(dec_col.data)))
    
if ra_range[1]<ra_range[0] or dec_range[1]<dec_range[0]:
    raise ValueError("Invalid range")

In [ ]:
# Select only those TU sources that fall inside range
star_catalog = select_true_universe_sources(star_catalog_file_names, ra_range, dec_range)
galaxy_catalog = select_true_universe_sources(galaxy_catalog_file_names, ra_range, dec_range)

# Save the True Universe catalogs in the work directory
star_catalog.write(os.path.join(workdir, "tu_star_catalog_v7_batch_%s.fits" % batch_number), format="fits", overwrite=True)
galaxy_catalog.write(os.path.join(workdir, "tu_galaxy_catalog_v8p1_batch_%s.fits" % batch_number), format="fits", overwrite=True)

In [ ]:


# Plot the True Universe star positions and the MER detected sources on top of the mosaic image
plt.figure()
plt.imshow(mosaic_data, origin="lower", vmin=vmin, vmax=vmax)
plt.colorbar()
plt.gca().set_xlim(auto=False)
plt.gca().set_ylim(auto=False)
plt.plot(star_catalog["MOSAIC_X"] - 1, star_catalog["MOSAIC_Y"] - 1, "r.", markersize=4)
plt.plot(catalog["MOSAIC_X"][to_plot] - 1, catalog["MOSAIC_Y"][to_plot] - 1, "ro", markersize=7, fillstyle="none")
plt.title("%s mosaic" % mosaic_xml.get_filter())
plt.xlabel("x (pixels)")
plt.ylabel("y (pixels)")
plt.show()